### Task: Using this sample dataset, create or optimize data preprocessing pipeline using opencv and pytorch and include image mask technique in the pipeline and save the output.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE'

In [ ]:
def preprocess_transform():
    # simple wrapper so it behaves like albumentations-style transform
    def apply(image, mask):
        return {
            "image": image,
            "mask": mask
        }
    return apply


In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
class CocoSampleDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        self.images = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
        self.masks  = sorted([f for f in os.listdir(mask_dir) if f.endswith((".png",".jpg"))])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        img_name = self.images[idx]
        mask_name = self.masks[idx]

        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)

        # read image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # read mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # resize
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)

        # binary mask
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        if self.transform:
            out = self.transform(image=image, mask=mask)
            image = out["image"]
            mask  = out["mask"]

        # convert to torch
        image = torch.from_numpy(image).permute(2,0,1).float() / 255.0
        mask  = torch.from_numpy(mask).long()

        return {"image": image, "mask": mask, "name": img_name}


In [ ]:
def save_output(sample, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    img = sample["image"].permute(1,2,0).numpy()*255
    img = img.astype(np.uint8)

    mask = sample["mask"].numpy().astype(np.uint8)

    base = sample["name"].split(".")[0]

    cv2.imwrite(os.path.join(out_dir, f"{base}_processed.png"), img)
    cv2.imwrite(os.path.join(out_dir, f"{base}_mask.png"), mask)

    torch.save(sample, os.path.join(out_dir, f"{base}.pt"))


In [ ]:
def run_pipeline(image_dir, mask_dir, out_dir):

    dataset = CocoSampleDataset(
        image_dir=image_dir,
        mask_dir=mask_dir,
        transform=preprocess_transform()
    )

    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    for batch in tqdm(loader):
        sample = {
            "image": batch["image"][0],
            "mask": batch["mask"][0],
            "name": batch["name"][0]
        }
        save_output(sample, out_dir)

    print("✔ Processing complete!")


In [ ]:
image_dir = "/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE"
mask_dir  = "/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE"
out_dir   = "/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE"

!mkdir -p "$out_dir"

run_pipeline(image_dir, mask_dir, out_dir)


100%|██████████| 5/5 [00:00<00:00, 11.04it/s]

✔ Processing complete!
